In [ ]:
import datetime
import panel as pn

pn.extension('tabulator', notifications=True)

import panel as pn
import matplotlib.pyplot as plt
from src.dao import pessoa_dao, plantio_dao, propriedade_dao, visita_dao

pn.extension('tabulator', notifications=True)

# Título do Sistema
titulo = pn.pane.Markdown("# 🚜 AgroGestão - Sistema Integrado")

In [ ]:
# CÉLULA 2 - GESTÃO DE PESSOAS (Produtor + Técnico)
import panel as pn
import matplotlib.pyplot as plt
from src.dao import pessoa_dao

# --- WIDGETS ---
w_pes_id = pn.widgets.IntInput(name="ID", disabled=True, visible=False)
w_pes_nome = pn.widgets.TextInput(name="Nome Completo", placeholder="Ex: Maria Silva")
w_pes_cpf = pn.widgets.TextInput(name="CPF", placeholder="000.000.000-00")
w_pes_nasc = pn.widgets.DatePicker(name="Data Nascimento")

# Widget de Busca (NOVO)
w_pes_busca = pn.widgets.TextInput(name="🔍 Buscar por Nome", placeholder="Digite para filtrar...")

# Checkboxes para definir o papel
w_check_prod = pn.widgets.Checkbox(name="É Produtor?", value=True)
w_check_tec = pn.widgets.Checkbox(name="É Técnico?", value=False)

# Campos Específicos
w_pes_insc = pn.widgets.TextInput(name="Inscrição Estadual (Produtor)")
w_pes_crea = pn.widgets.TextInput(name="CREA (Técnico)")

# Lógica de Visibilidade
def toggle_produtor(event):
    w_pes_insc.visible = event.new
    w_pes_insc.disabled = not event.new

def toggle_tecnico(event):
    w_pes_crea.visible = event.new
    w_pes_crea.disabled = not event.new

w_check_prod.param.watch(toggle_produtor, 'value')
w_check_tec.param.watch(toggle_tecnico, 'value')

# Estado inicial
w_pes_insc.visible = True
w_pes_crea.visible = False

# Botões e Tabela
btn_pes_salvar = pn.widgets.Button(name='💾 Salvar Pessoa', button_type='success')
btn_pes_limpar = pn.widgets.Button(name='🧹 Limpar', button_type='default')
btn_pes_excluir = pn.widgets.Button(name='🗑️ Excluir', button_type='danger', disabled=True)

tab_pes = pn.widgets.Tabulator(show_index=False, sizing_mode='stretch_width', page_size=5)
graf_pes = pn.pane.Matplotlib(tight=True)

# --- LÓGICA DE DADOS ---

async def carregar_pessoas(event=None):
    # Passamos o valor da busca para o DAO
    termo_busca = w_pes_busca.value
    tab_pes.value = await pessoa_dao.listar(termo_busca)
    
    # Gráfico
    df_g = await pessoa_dao.dados_grafico_papeis()
    if not df_g.empty:
        fig, ax = plt.subplots(figsize=(5, 3))
        ax.bar(df_g['Tipo'], df_g['Total'], color=['#4CAF50', '#2196F3'])
        ax.set_title("Distribuição de Cadastros")
        plt.close(fig)
        graf_pes.object = fig

def selecionar_pessoa(event):
    if event.row is None: return
    try:
        # Correção do clique (usando iloc)
        row_index = event.row
        df = tab_pes.value
        row = df.iloc[row_index]

        w_pes_id.value = int(row['id'])
        w_pes_nome.value = str(row['Nome'])
        w_pes_cpf.value = str(row['CPF'])
        
        # Data
        if row['Nascimento']:
            val = row['Nascimento']
            w_pes_nasc.value = val.date() if hasattr(val, 'date') else val
            
        # Papéis
        is_prod = bool(row['É Produtor?'])
        is_tec = bool(row['É Técnico?'])
        
        w_check_prod.value = is_prod
        w_check_tec.value = is_tec
        
        if is_prod: w_pes_insc.value = str(row['Inscrição Est.'])
        else: w_pes_insc.value = ""
            
        if is_tec: w_pes_crea.value = str(row['CREA']) if row['CREA'] else ""
        else: w_pes_crea.value = ""

        btn_pes_salvar.name = "🔄 Atualizar"
        btn_pes_salvar.button_type = "warning"
        btn_pes_excluir.disabled = False
        
        pn.state.notifications.info(f"Editando: {row['Nome']}")
    except Exception as e:
        print(f"Erro ao selecionar: {e}")

async def salvar_pessoa(event):
    try:
        if not w_pes_nome.value or not w_pes_cpf.value:
            pn.state.notifications.error("Nome e CPF obrigatórios!")
            return
            
        if not w_check_prod.value and not w_check_tec.value:
             pn.state.notifications.warning("Selecione pelo menos um papel (Produtor ou Técnico)!")
             return

        btn_pes_salvar.loading = True
        
        await pessoa_dao.salvar(
            w_pes_id.value if w_pes_id.value > 0 else None,
            w_pes_nome.value, w_pes_cpf.value, w_pes_nasc.value,
            w_check_prod.value, w_pes_insc.value,
            w_check_tec.value, w_pes_crea.value
        )
        
        pn.state.notifications.success("Salvo com sucesso!")
        limpar_pessoa(None)
        await carregar_pessoas()
        
    except Exception as e:
        pn.state.notifications.error(f"Erro: {e}")
    finally:
        btn_pes_salvar.loading = False

async def excluir_pessoa(event):
    if w_pes_id.value:
        await pessoa_dao.excluir(w_pes_id.value)
        pn.state.notifications.success("Pessoa excluída!")
        limpar_pessoa(None)
        await carregar_pessoas()

def limpar_pessoa(event):
    w_pes_id.value = 0
    w_pes_nome.value = ""
    w_pes_cpf.value = ""
    w_pes_insc.value = ""
    w_pes_crea.value = ""
    w_check_prod.value = True
    w_check_tec.value = False
    btn_pes_salvar.name = "💾 Salvar Pessoa"
    btn_pes_salvar.button_type = "success"
    btn_pes_excluir.disabled = True

# --- LIGAÇÕES ---
btn_pes_salvar.on_click(salvar_pessoa)
btn_pes_limpar.on_click(limpar_pessoa)
btn_pes_excluir.on_click(excluir_pessoa)
tab_pes.on_click(selecionar_pessoa)

# Liga a busca: ao digitar, recarrega a tabela automaticamente
w_pes_busca.param.watch(carregar_pessoas, 'value')

# --- LAYOUT ---
layout_produtor = pn.Row(
    pn.Column(
        "### Dados Pessoais", 
        w_pes_nome, w_pes_cpf, w_pes_nasc,
        "### Papéis",
        pn.Row(w_check_prod, w_check_tec),
        w_pes_insc, w_pes_crea,
        pn.Row(btn_pes_salvar, btn_pes_limpar, btn_pes_excluir)
    ),
    pn.Column(
        "### Base de Pessoas",
        w_pes_busca,  # Campo de busca adicionado aqui
        tab_pes, 
        "### Estatísticas",
        graf_pes
    )
)

# Carrega ao iniciar
pn.state.onload(carregar_pessoas)

In [ ]:
# CÉLULA 3 - PROPRIEDADE (Samuel)
import panel as pn
import matplotlib.pyplot as plt
from src.dao import propriedade_dao 

# --- WIDGETS ---
w_prop_id = pn.widgets.IntInput(name="ID", disabled=True, visible=False)

# Campos de Cadastro
w_prop_nome = pn.widgets.TextInput(name="Nome da Propriedade", placeholder="Ex: Fazenda Esperança")
w_prop_reg = pn.widgets.TextInput(name="Registro do Imóvel (RGI)", placeholder="Ex: RGI-1234")
w_prop_area = pn.widgets.FloatInput(name="Área Total (ha)", value=0.0, step=1.0)
w_prop_log = pn.widgets.TextInput(name="Logradouro", placeholder="Rua...")
w_prop_num = pn.widgets.TextInput(name="Número", placeholder="S/N")
w_prop_cep = pn.widgets.TextInput(name="CEP", placeholder="00000000")
w_sel_mun = pn.widgets.Select(name="Município")

# Widget de Busca (NOVO)
w_prop_busca = pn.widgets.TextInput(name="🔍 Buscar Propriedade", placeholder="Digite o nome da fazenda...")

# Botões
btn_prop_salvar = pn.widgets.Button(name='🏡 Salvar Propriedade', button_type='primary')
btn_prop_limpar = pn.widgets.Button(name='🧹 Limpar', button_type='default')
btn_prop_excluir = pn.widgets.Button(name='🗑️ Excluir', button_type='danger', disabled=True)

tab_prop = pn.widgets.Tabulator(show_index=False, sizing_mode='stretch_width', page_size=5)
graf_prop = pn.pane.Matplotlib(tight=True)

# --- LÓGICA ---
async def carregar_propriedades(event=None):
    try:
        # Preenche o Select (caso tenha novos municípios)
        w_sel_mun.options = await propriedade_dao.get_opcoes_municipio()
        
        # Preenche a Tabela (COM FILTRO)
        termo_busca = w_prop_busca.value
        tab_prop.value = await propriedade_dao.listar(termo_busca)
        
        # Preenche o Gráfico
        df_g = await propriedade_dao.dados_grafico_municipio()
        if not df_g.empty:
            fig, ax = plt.subplots(figsize=(5, 3))
            ax.bar(df_g['Município'], df_g['Total Hectares'], color='orange')
            ax.set_title("Top 5 Municípios por Área")
            plt.close(fig)
            graf_prop.object = fig
    except Exception as e:
        pn.state.notifications.error(f"Erro ao carregar: {e}")

def selecionar_propriedade(event):
    if event.row is None: return
    try:
        # Correção do clique usando iloc
        df = tab_prop.value
        row = df.iloc[event.row]
        
        w_prop_id.value = int(row['id'])
        w_prop_nome.value = str(row['Nome'])
        w_prop_reg.value = str(row['Registro'])
        w_prop_area.value = float(row['Área (ha)']) if 'Área (ha)' in row else 0.0
        w_prop_log.value = str(row['Logradouro'])
        w_prop_num.value = str(row['Num'])
        w_prop_cep.value = str(row['CEP'])
        
        if 'id_mun_oculto' in row:
             try: w_sel_mun.value = int(row['id_mun_oculto'])
             except: pass

        btn_prop_salvar.name = "🔄 Atualizar"
        btn_prop_salvar.button_type = "warning"
        btn_prop_excluir.disabled = False
        
        pn.state.notifications.info(f"Editando: {row['Nome']}")
    except Exception as e:
        print(f"Erro seleção: {e}")

async def salvar_propriedade(event):
    try:
        if not w_prop_nome.value or not w_sel_mun.value:
            pn.state.notifications.error("Preencha Nome e Município!")
            return
        
        btn_prop_salvar.loading = True

        if w_prop_id.value:
            await propriedade_dao.atualizar(
                w_prop_id.value, w_prop_nome.value, w_prop_reg.value, w_prop_area.value,
                w_prop_log.value, w_prop_num.value, w_prop_cep.value, w_sel_mun.value
            )
            pn.state.notifications.success("Atualizado!")
        else:
            await propriedade_dao.inserir(
                w_prop_nome.value, w_prop_reg.value, w_prop_area.value,
                w_prop_log.value, w_prop_num.value, w_prop_cep.value, w_sel_mun.value
            )
            pn.state.notifications.success("Salvo!")
            
        limpar_propriedade(None)
        await carregar_propriedades()
    except Exception as e:
        pn.state.notifications.error(f"Erro ao salvar: {e}")
    finally:
        btn_prop_salvar.loading = False

async def excluir_propriedade(event):
    if w_prop_id.value:
        await propriedade_dao.excluir(w_prop_id.value)
        pn.state.notifications.success("Excluído!")
        limpar_propriedade(None)
        await carregar_propriedades()

def limpar_propriedade(event):
    w_prop_id.value = 0
    w_prop_nome.value = ""
    w_prop_reg.value = ""
    w_prop_area.value = 0.0
    w_prop_log.value = ""
    w_prop_num.value = ""
    w_prop_cep.value = ""
    btn_prop_salvar.name = "🏡 Salvar Propriedade"
    btn_prop_salvar.button_type = "primary"
    btn_prop_excluir.disabled = True

# --- LIGAÇÕES ---
btn_prop_salvar.on_click(salvar_propriedade)
btn_prop_limpar.on_click(limpar_propriedade)
btn_prop_excluir.on_click(excluir_propriedade)
tab_prop.on_click(selecionar_propriedade)

# Liga o campo de busca à função de carregar
w_prop_busca.param.watch(carregar_propriedades, 'value')

# --- LAYOUT ---
layout_propriedade = pn.Row(
    pn.Column(
        "### Nova Propriedade", 
        w_prop_nome, w_prop_reg, w_prop_area, 
        w_sel_mun, w_prop_log, w_prop_num, w_prop_cep, 
        pn.Row(btn_prop_salvar, btn_prop_limpar, btn_prop_excluir)
    ),
    pn.Column(
        "### Lista de Propriedades", 
        w_prop_busca, # Campo de busca adicionado aqui
        tab_prop, 
        "### Área por Município",
        graf_prop
    )
)

# Carrega ao iniciar
pn.state.onload(carregar_propriedades)

In [ ]:
# CÉLULA 4 - VISITA TÉCNICA

# --- WIDGETS ---
w_visita_id = pn.widgets.IntInput(name="ID", disabled=True, visible=False)

# Selects
w_sel_tec_visita = pn.widgets.Select(name="Técnico Responsável")
w_sel_prop_visita = pn.widgets.Select(name="Propriedade Visitada")

# Campos de Dados
w_visita_data = pn.widgets.DatePicker(name="Data da Visita", value=datetime.date.today())
w_visita_obs = pn.widgets.TextAreaInput(name="Observações / Laudo", placeholder="Descreva o que foi analisado...", height=100)

# Busca
w_visita_busca = pn.widgets.TextInput(name="🔍 Buscar nas Observações", placeholder="Ex: praga, adubação...")

# Botões
btn_visita_salvar = pn.widgets.Button(name='📋 Registrar Visita', button_type='success')
btn_visita_limpar = pn.widgets.Button(name='🧹 Limpar', button_type='default')
btn_visita_excluir = pn.widgets.Button(name='🗑️ Excluir', button_type='danger', disabled=True)

tab_visita = pn.widgets.Tabulator(show_index=False, sizing_mode='stretch_width', page_size=5)
graf_visita = pn.pane.Matplotlib(tight=True)

# --- LÓGICA ---

async def carregar_opcoes_visita():
    w_sel_tec_visita.options = await visita_dao.get_opcoes_tecnico()
    w_sel_prop_visita.options = await visita_dao.get_opcoes_propriedade()

async def carregar_visitas(event=None):
    try:
        await carregar_opcoes_visita()
        
        termo = w_visita_busca.value
        tab_visita.value = await visita_dao.listar(termo)
        
        # Gráfico
        df_g = await visita_dao.dados_grafico_visitas()
        if not df_g.empty:
            fig, ax = plt.subplots(figsize=(5, 3))
            ax.barh(df_g['Técnico'], df_g['Total Visitas'], color='#9C27B0') # Roxo
            ax.set_title("Visitas por Técnico")
            plt.close(fig)
            graf_visita.object = fig
            
    except Exception as e:
        pn.state.notifications.error(f"Erro ao carregar visitas: {e}")

def selecionar_visita(event):
    if event.row is None: return
    try:
        row = tab_visita.value.iloc[event.row]
        
        w_visita_id.value = int(row['id'])
        w_visita_obs.value = str(row['Observações'])
        
        # Data
        if row['Data']:
            val = row['Data']
            w_visita_data.value = val.date() if hasattr(val, 'date') else val
            
        # Selects (Tentativa de preencher pelos IDs ocultos)
        if 'id_tec_oculto' in row:
             try: w_sel_tec_visita.value = int(row['id_tec_oculto'])
             except: pass
             
        if 'id_prop_oculto' in row:
             try: w_sel_prop_visita.value = int(row['id_prop_oculto'])
             except: pass

        btn_visita_salvar.name = "🔄 Atualizar Visita"
        btn_visita_salvar.button_type = "warning"
        btn_visita_excluir.disabled = False
        
        pn.state.notifications.info(f"Editando visita #{row['id']}")
    except Exception as e:
        print(f"Erro seleção: {e}")

async def salvar_visita(event):
    try:
        if not w_sel_tec_visita.value or not w_sel_prop_visita.value:
            pn.state.notifications.error("Selecione o Técnico e a Propriedade!")
            return

        btn_visita_salvar.loading = True
        
        await visita_dao.salvar(
            w_visita_id.value if w_visita_id.value > 0 else None,
            w_sel_tec_visita.value,
            w_sel_prop_visita.value,
            w_visita_data.value,
            w_visita_obs.value
        )
        
        pn.state.notifications.success("Visita registrada!")
        limpar_visita(None)
        await carregar_visitas()
        
    except Exception as e:
        pn.state.notifications.error(f"Erro: {e}")
    finally:
        btn_visita_salvar.loading = False

async def excluir_visita(event):
    if w_visita_id.value:
        await visita_dao.excluir(w_visita_id.value)
        pn.state.notifications.success("Visita excluída!")
        limpar_visita(None)
        await carregar_visitas()

def limpar_visita(event):
    w_visita_id.value = 0
    w_visita_obs.value = ""
    w_visita_data.value = datetime.date.today()
    btn_visita_salvar.name = "📋 Registrar Visita"
    btn_visita_salvar.button_type = "success"
    btn_visita_excluir.disabled = True

# --- LIGAÇÕES ---
btn_visita_salvar.on_click(salvar_visita)
btn_visita_limpar.on_click(limpar_visita)
btn_visita_excluir.on_click(excluir_visita)
tab_visita.on_click(selecionar_visita)
w_visita_busca.param.watch(carregar_visitas, 'value')

# --- LAYOUT ---
layout_visita = pn.Row(
    pn.Column(
        "### 📋 Nova Visita", 
        w_sel_tec_visita, 
        w_sel_prop_visita, 
        w_visita_data, 
        w_visita_obs, 
        pn.Row(btn_visita_salvar, btn_visita_limpar, btn_visita_excluir)
    ),
    pn.Column(
        "### Histórico de Visitas", 
        w_visita_busca,
        tab_visita, 
        "### Produtividade da Equipa",
        graf_visita
    )
)

# Carrega ao iniciar
pn.state.onload(carregar_visitas)

In [ ]:
# CÉLULA 5 - GESTÃO DE PLANTIO

# --- WIDGETS ---
w_plant_id = pn.widgets.IntInput(name="ID", disabled=True, visible=False)

# Busca Automática (Sem botão)
w_busca_plantio = pn.widgets.TextInput(name="🔍 Buscar Plantio", placeholder="Digite Safra ou Ano (ex: 2025)...")

# Selects (Carregados do Banco)
w_sel_safra = pn.widgets.Select(name="Safra")
w_sel_cultivo = pn.widgets.Select(name="Cultivo/Variedade")
w_sel_prop = pn.widgets.Select(name="Propriedade")

w_data_plantio = pn.widgets.DatePicker(name="Data do Plantio", value=datetime.date.today())
w_area_plantada = pn.widgets.FloatInput(name="Área Plantada (Hectares)", value=0.0, step=0.1)

btn_plant_salvar = pn.widgets.Button(name='🌱 Registrar Plantio', button_type='success')
btn_plant_limpar = pn.widgets.Button(name='🧹 Limpar', button_type='default')
btn_plant_excluir = pn.widgets.Button(name='🗑️ Excluir', button_type='danger', disabled=True)

tab_plant = pn.widgets.Tabulator(show_index=False, sizing_mode='stretch_width', page_size=5)
graf_plant = pn.pane.Matplotlib(tight=True)

# --- LÓGICA ---

async def carregar_opcoes_plantio():
    """Preenche os menus com dados do banco"""
    w_sel_safra.options = await plantio_dao.get_opcoes_safra()
    w_sel_cultivo.options = await plantio_dao.get_opcoes_cultivo()
    w_sel_prop.options = await plantio_dao.get_opcoes_propriedade()

async def carregar_plantios(event=None):
    try:
        # 1. Carrega opções
        await carregar_opcoes_plantio()

        # 2. Carrega Tabela (COM O FILTRO)
        termo = w_busca_plantio.value 
        tab_plant.value = await plantio_dao.listar(termo) 
        
        # 3. Carrega Gráfico
        df_g = await plantio_dao.dados_grafico_cultura()
        if not df_g.empty:
            fig, ax = plt.subplots(figsize=(5, 3))
            ax.pie(df_g['Total Hectares'], labels=df_g['Cultura'], autopct='%1.1f%%', startangle=90, colors=['#8dd3c7','#ffffb3','#bebada','#fb8072'])
            ax.set_title("Área Plantada por Cultura")
            plt.close(fig)
            graf_plant.object = fig
    except Exception as e:
        pn.state.notifications.error(f"Erro ao carregar plantios: {e}")

def selecionar_plantio(event):
    if event.row is None: return

    try:
        row_index = event.row
        df = tab_plant.value
        row = df.iloc[row_index]
        
        w_plant_id.value = int(row['id'])
        
        if 'Área (ha)' in row:
            w_area_plantada.value = float(row['Área (ha)'])
            
        if 'Data Plantio' in row and row['Data Plantio']:
            val = row['Data Plantio']
            if hasattr(val, 'date'):
                w_data_plantio.value = val.date()
            else:
                w_data_plantio.value = val 
        
        btn_plant_salvar.name = "🔄 Atualizar Dados"
        btn_plant_salvar.button_type = "warning"
        btn_plant_excluir.disabled = False
        
        pn.state.notifications.info(f"Editando Plantio #{row['id']}")

    except Exception as e:
        print(f"Erro: {e}")

async def salvar_plantio(event):
    try:
        if not w_sel_safra.value or not w_sel_cultivo.value or not w_sel_prop.value:
            pn.state.notifications.error("Selecione Safra, Cultivo e Propriedade!")
            return
        if w_area_plantada.value <= 0:
            pn.state.notifications.error("A área deve ser maior que zero!")
            return

        btn_plant_salvar.loading = True
        
        if w_plant_id.value: 
            await plantio_dao.atualizar(
                w_plant_id.value, w_sel_safra.value, w_sel_cultivo.value, 
                w_sel_prop.value, w_data_plantio.value, w_area_plantada.value
            )
            pn.state.notifications.success("Plantio atualizado!")
        else: 
            await plantio_dao.inserir(
                w_sel_safra.value, w_sel_cultivo.value, 
                w_sel_prop.value, w_data_plantio.value, w_area_plantada.value
            )
            pn.state.notifications.success("Plantio registrado!")

        limpar_plantio(None)
        await carregar_plantios()
    except Exception as e:
        pn.state.notifications.error(f"Erro: {e}")
    finally:
        btn_plant_salvar.loading = False

async def excluir_plantio(event):
    if w_plant_id.value:
        try:
            await plantio_dao.excluir(w_plant_id.value)
            pn.state.notifications.success("Plantio excluído!")
            limpar_plantio(None)
            await carregar_plantios()
        except Exception as e:
            pn.state.notifications.error(f"Erro ao excluir: {e}")

def limpar_plantio(event):
    w_plant_id.value = 0
    w_data_plantio.value = datetime.date.today()
    w_area_plantada.value = 0.0
    btn_plant_salvar.name = "🌱 Registrar Plantio"
    btn_plant_salvar.button_type = "success"
    btn_plant_excluir.disabled = True

# --- LIGAÇÕES ---
btn_plant_salvar.on_click(salvar_plantio)
btn_plant_limpar.on_click(limpar_plantio)
btn_plant_excluir.on_click(excluir_plantio)
tab_plant.on_click(selecionar_plantio)

# LIGAÇÃO DA BUSCA AUTOMÁTICA
w_busca_plantio.param.watch(carregar_plantios, 'value')

# --- LAYOUT ---
layout_plantio = pn.Row(
    pn.Column(
        "### 🚜 Novo Plantio", 
        w_sel_safra, 
        w_sel_cultivo, 
        w_sel_prop, 
        w_data_plantio, 
        w_area_plantada, 
        pn.Row(btn_plant_salvar, btn_plant_limpar, btn_plant_excluir)
    ),
    pn.Column(
        "### Histórico de Plantios",
        w_busca_plantio, # Busca automática aqui
        tab_plant, 
        "### Estatísticas (Área)",
        graf_plant
    )
)

# Carrega ao iniciar
pn.state.onload(carregar_plantios)

In [ ]:
# CÉLULA 6 - DASHBOARD FINAL

# Função para carregar tudo ao abrir
async def inicializar_tudo():
    await carregar_pessoas()
    await carregar_propriedades()
    await carregar_plantios()
    await carregar_visitas()
    
pn.state.onload(inicializar_tudo)

# Cria as Abas Navegáveis
dashboard = pn.Tabs(
    ("🧑‍🌾 Pessoas", layout_produtor),
    ("📋 Visitas", layout_visita),
    ("🏡 Propriedades", layout_propriedade),
    ("🌱 Plantios", layout_plantio)
)

# Exibe o Dashboard Completo
pn.Column(
    titulo,
    dashboard
).servable()